In [1]:
import os
import sys

from typing import List
import numpy as np
import cv2


In [2]:
def load_list(fname: str):
    """Plain text list loader. Reads from file separated by newlines, and returns a
    list of the file with whitespaces stripped.
    Args:
        fname (str): Name of file to be read.
    Returns:
        List[str]: A stripped list of strings, using newlines as a seperator from file.
    """

    return [e.strip() for e in open(fname, 'r').readlines()]

In [3]:
import re

In [4]:
#query_file_name = "radcliffe_camera_1_query.txt"
def cropImage(query_file_name: str, groundtruthFolderDir: str, imgFolderDir: str, outputFolderDir: str):
    print(query_file_name)
    query_key = query_file_name.replace("_query.txt", "")
    query_detail = load_list(groundtruthFolderDir + "/" + query_file_name)
    list = query_detail[0].split()
    imgName = list[0].replace("oxc1_", "")
    img = cv2.imread(imgFolderDir + "/%s.jpg" % imgName)
    x1 = int(float(list[1]))
    y1 = int(float(list[2]))
    x2 = int(float(list[3]))
    y2 = int(float(list[4]))
    crop_img = img[y1:y2, x1:x2]
    print(outputFolderDir)
    cv2.imwrite(outputFolderDir + "/%s.jpg" % query_key, crop_img)


In [5]:
datasetDir="./static/images/database_paris"
evalGroundTruth="./evaluation/paris_gt"
evalQueryImg="./query_img/paris"

In [6]:
folder = evalGroundTruth
names = []
for basename in os.listdir(folder):
  name = os.path.splitext(basename)[0]
  matches = re.findall(".+_query", name)
  if len(matches) > 0:
    names.append(name)

for name in names:
  fileName = name + ".txt"
  print(fileName)
  cropImage(fileName, folder, datasetDir, evalQueryImg)

defense_1_query.txt
defense_1_query.txt
./query_img/paris
defense_2_query.txt
defense_2_query.txt
./query_img/paris
defense_3_query.txt
defense_3_query.txt
./query_img/paris
defense_4_query.txt
defense_4_query.txt
./query_img/paris
defense_5_query.txt
defense_5_query.txt
./query_img/paris
eiffel_1_query.txt
eiffel_1_query.txt
./query_img/paris
eiffel_2_query.txt
eiffel_2_query.txt
./query_img/paris
eiffel_3_query.txt
eiffel_3_query.txt
./query_img/paris
eiffel_4_query.txt
eiffel_4_query.txt
./query_img/paris
eiffel_5_query.txt
eiffel_5_query.txt
./query_img/paris
invalides_1_query.txt
invalides_1_query.txt
./query_img/paris
invalides_2_query.txt
invalides_2_query.txt
./query_img/paris
invalides_3_query.txt
invalides_3_query.txt
./query_img/paris
invalides_4_query.txt
invalides_4_query.txt
./query_img/paris
invalides_5_query.txt
invalides_5_query.txt
./query_img/paris
louvre_1_query.txt
louvre_1_query.txt
./query_img/paris
louvre_2_query.txt
louvre_2_query.txt
./query_img/paris
louvre_3

In [7]:
def prepareValidationMaterial(grouthtruthUrl ,query):
  pos_set = list(set(load_list(grouthtruthUrl + "/%s_good.txt" % query) + load_list(grouthtruthUrl + "/%s_ok.txt" % query)))
  junk_set = load_list(grouthtruthUrl + "/%s_junk.txt" % query)

  return pos_set, junk_set

In [8]:
from extraction.VGG16 import DeepVGG16
feature_extractor=DeepVGG16()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
from retrieval import retrieval_image

In [10]:
feature_extractor=DeepVGG16()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
def compute_ap(pos: List[str], amb: List[str], ranked_list: List[str]):
    """Compute average precision against a retrieved list of images. There are some bits that
    could be improved in this, but is a line-to-line port of the original C++ benchmark code.
    Args:
        pos (List[str]): List of positive samples. This is normally a conjugation of
        the good and ok samples in the ground truth data.
        amb (List[str]): List of junk samples. This is normally the junk samples in
        the ground truth data. Omitting this makes no difference in the AP.
        ranked_list (List[str]): List of retrieved images from query to be evaluated.
    Returns:
        float: Average precision against ground truth - range from 0.0 (worst) to 1.0 (best).
    """

    intersect_size, old_recall, ap = 0.0, 0.0, 0.0
    old_precision, j = 1.0, 1.0

    for e in ranked_list:
        if e in amb:
            continue

        if e in pos:
            intersect_size += 1.0

        recall = intersect_size / len(pos)
        precision = intersect_size / j
        ap += (recall - old_recall) * ((old_precision + precision) / 2.0)

        old_recall = recall
        old_precision = precision
        j += 1.0

    return ap

In [12]:
def compute_ap_at_k(pos: List[str], amb: List[str], ranked_list: List[str], k):
    """Compute average precision against a retrieved list of images. There are some bits that
    could be improved in this, but is a line-to-line port of the original C++ benchmark code.
    Args:
        pos (List[str]): List of positive samples. This is normally a conjugation of
        the good and ok samples in the ground truth data.
        amb (List[str]): List of junk samples. This is normally the junk samples in
        the ground truth data. Omitting this makes no difference in the AP.
        ranked_list (List[str]): List of retrieved images from query to be evaluated.
    Returns:
        float: Average precision against ground truth - range from 0.0 (worst) to 1.0 (best).
    """

    intersect_size, old_recall, ap = 0.0, 0.0, 0.0
    old_precision, j = 1.0, 1.0

    for e in ranked_list:
      if j <= k:
        if e in amb:
            continue

        if e in pos:
            intersect_size += 1.0

        recall = intersect_size / len(pos)
        precision = intersect_size / j
        ap += (recall - old_recall) * ((old_precision + precision) / 2.0)

        old_recall = recall
        old_precision = precision
        j += 1.0

    return ap

In [13]:
# from retrieval import retrieve_img_vgg16

In [14]:
# retrieve_img_vgg16

In [15]:
img_name=os.listdir("./static/images/database_paris")

In [18]:
aps = []
for query_img_path in os.listdir("./query_img/paris"):
    name = os.path.splitext(query_img_path)[0]
    ids=retrieval_image(os.path.join("./query_img/paris", query_img_path), "delf", "paris")

    ranked_list = []
    for item in ids:
        ranked_list.append(item[2].split("/")[len(item[2].split("/"))-1][:-5])
    
    pos_set, junk_set = prepareValidationMaterial(evalGroundTruth, name)
    # print(pos_set)
    # print(ranked_list)
    ap = compute_ap(pos_set, junk_set, ranked_list)
    print(ap)
    #print(junk_set)
    #print(name)
    aps.append(ap)

      
print(np.mean(aps))

40
0.43407187588599583
40
0.4071577811174915
40
0.39316239316239315
40
0.35960906082754773
40
0.39316239316239315
40
0.11268021408096025
40
0.18158027356730624
40
0.15434359336473957
40
0.191179586603269
40
0.09687232641927017
40
0.3403359017612596
40
0.30925779925968155
40
0.3034528207388328
40
0.24744131271198258
40
0.3351962394548779
40
0.191951615701434
40
0.19620173995772924
40
0.18554079747056965
40
0.23699415536237928
40
0.25312790093480353
40
0.21940928270042195
40
0.29957805907172996
40
0.25316455696202533
40
0.18143459915611815
40
0.19831223628691982
40
0.33155733055189573
40
0.3292273187427172
40
0.3134826329541718
40
0.2939779982405027
40
0.3385529264099357
40
0.6176655981513732
40
0.5739646487858238
40
0.5655779237681353
40
0.5542368796882077
40
0.6615868727337622
40
0.5048395162346617
40
0.5
40
0.38483424703114527
40
0.42857142857142855
40
0.5079365079365079
40
0.5686274509803921
40
0.5098039215686274
40
0.8034490218851769
40
0.6176292734618755
40
0.567962587333784
40
0.6

In [ ]:
def cal_mAP_at_k(k): #10, 20, 30
    aps = []
    for query_img_path in os.listdir("./query_img"):
        name = os.path.splitext(query_img_path)[0]
        ids=retrieval_image(os.path.join("./query_img",query_img_path), "delf")
        pos_set, junk_set = prepareValidationMaterial(evalGroundTruth, name)
        #print(pos_set)
        ranked_list = []
        for item in ids:
            ranked_list.append(item[2].split("/")[len(item[2].split("/"))-1][:-5])
        #print(ranked_list)
        ap=compute_ap_at_k(pos_set, junk_set, ranked_list,k)
        print(ap)
        #print(junk_set)
        #print(name)
        aps.append(ap)

        
    print(np.mean(aps))
    return np.mean(aps)

In [ ]:
# cal_mAP_at_k(10)

40
0.11538461538461539
40


KeyboardInterrupt: 

In [ ]:
# cal_mAP_at_k(20)

TypeError: retrieval_image() missing 1 required positional argument: 'method'

In [ ]:
# cal_mAP_at_k(30)

In [ ]:
# print(np.mean(aps))#map

0.269006339589123
